# Trabajo Pracatico 4 - Redes Neuronales Convolucionales

Vamos a realizar una clasificacion binaria de imagnes que pueden ser de gatos o perros, para ello vamos a utilizar un dataset de Kaggle llamado "cats-vs-dogs" que contiene 23.409 imágenes de gatos y perros. El objetivo es entrenar un modelo de clasificación binaria que pueda distinguir entre imágenes de gatos y perros.

Se van a proponer los siguientes modelos:

- **Modelo 1:** Red convolucional simple (SimpleCNN) creada desde cero.
- **Modelo 2:** Red convolucional basada en ResNet18.
- **Modelo 4:** Red convolucional avanzacda (AdvancedCNN) creada desde cero.
- **Modelo 3:** Red convolucional basada en ResNet18 con cambios en hiperparametros.
- **Modelo 5:** Red convolucional basada en Inception de Google con cambios en hiperparametros.

Asignamos el dataset a la variable **dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cats_vs_dogs")

Creamos un *DataFrame* llamado **mydataset**, el cual almacenará el path de cada imágen junto a su etiqueta (perro o gato). Además creamos un directorio llamado dataset y almacenamos allí las imágenes.


In [ ]:
import pandas as pd
import os

main_dir = './dataset'
os.makedirs(main_dir, exist_ok=True)

mydataset = pd.DataFrame(columns=['image_path', 'label'])

for i in range(len(dataset['train'])):
    img_path = f"{main_dir}/img_{i}.jpeg"

    if not os.path.exists(img_path):
        dataset['train'][i]['image'].save(img_path)

    mydataset.at[i, 'image_path'] = img_path
    mydataset.at[i, 'label'] = dataset['train'][i]['labels']

mydataset.head()

Creamos un diccionario para almacenar los parámetros que usaremos.

In [ ]:
exp_config = dict()

Definimos la semilla para que al divir el dataset en train, test y val, sea siempre la misma división de datos. Además, especificamos la proporción de datos que serán para testeo y para validación.

In [ ]:
seed = 42
test_size = 0.15
val_size = 0.20

exp_config['seed'] = seed
exp_config['test_size'] = test_size
exp_config['val_size'] = val_size

Dividimos el dataset en *train*, *test*, *val*.

**Aclaración:** los datos de validación surgen de una parte de los datos de testeo.

In [ ]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(mydataset, test_size=test_size, stratify=mydataset['label'], random_state=seed)

train_df, val_df = train_test_split(train_val_df, test_size=val_size, stratify=train_val_df['label'], random_state=seed)

Añadimos parámetros de configuración al diccionario.

In [ ]:
exp_config['train_n_cats'] = train_df['label'].value_counts()[0]
exp_config['train_n_dogs'] = train_df['label'].value_counts()[1]
exp_config['val_n_cats'] = val_df['label'].value_counts()[0]
exp_config['val_n_dogs'] = val_df['label'].value_counts()[1]
exp_config['test_n_cats'] = test_df['label'].value_counts()[0]
exp_config['test_n_dogs'] = test_df['label'].value_counts()[1]

La clase **CatsDogsDataset** es una implementación personalizada de una clase llamda *Dataset* de PyTorch que permite cargar y transformar las imágenes del dataset.

**Explicación**
1. Constructor (\_\_init\_\_):  
- img_path_list: Lista de rutas de las imágenes.
- lab_list: Lista de etiquetas correspondientes a las imágenes (0 para gatos, 1 para perros).
- transform: Transformaciones opcionales que se aplicarán a las imágenes (por ejemplo, redimensionar, normalizar).
2. Método \_\_len\_\_:  
- Devuelve la cantidad de imágenes en el conjunto de datos.
3. Método \_\_getitem\_\_:
- idx: Índice de la imagen y etiqueta que se desea obtener.
- img_path: Obtiene la ruta de la imagen en el índice idx.
- image: Abre la imagen y la convierte a formato RGB.
- label: Obtiene la etiqueta correspondiente a la imagen y la convierte a un tensor de PyTorch.
- Si se especificaron transformaciones, se aplican a la imagen.
- Devuelve la imagen transformada y su etiqueta correspondiente.

In [ ]:
from PIL import Image
import torch
from torch.utils.data import Dataset

class CatsDogsDataset(Dataset):
    def __init__(self, img_path_list, lab_list, transform=None):
        self.transform = transform
        self.images = img_path_list
        self.labels = lab_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")

        label = self.labels[idx]
        label = torch.Tensor([label])

        if self.transform:
            image = self.transform(image)

        return image, label

Definimos la resolución de las imágenes que serán procesadas.

In [ ]:
input_size = (224,224)
exp_config['input_size'] = input_size

Como las imágenes son a color en formato RGB, definiremos 3 canales

In [ ]:
n_channels = 3
exp_config['n_channels'] = n_channels

Creamos el *transform* que será usado, el cual redimensiona las imágenes a la resolución dada.

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
])

Creamos los datasets de train, test y val.

In [ ]:
train_dataset = CatsDogsDataset(train_df['image_path'].tolist(), train_df['label'].tolist(), transform)
test_dataset = CatsDogsDataset(test_df['image_path'].tolist(), test_df['label'].tolist(), transform)
val_dataset = CatsDogsDataset(val_df['image_path'].tolist(), val_df['label'].tolist(), transform)

Creamos los *DataLoaders* de train, test y val, y definimos el tamaño de lote.

**Aclaración:** el batch size de test es 1,los datos no serán mezclados por cada época y no se eliminarán datos para alcanzar el tamaño de lote establecido.

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64
exp_config['batch_size'] = batch_size

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)

## WandB

In [ ]:
import wandb

wandb.login(key="d567fa512c6502cc7986d8c90fd37c4f0969de0d")

# Uso de los CNNs

In [ ]:
from tools.train_val_test import run
from tools.models import SimpleCNN, ResNet18, AdvancedCNN, DenseNet_121CNN

## SimpleCNN

In [12]:
from torch import nn
from torch import optim

model = SimpleCNN()

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)

'SimpleCNN'

## ResNet18

In [ ]:
from torch import nn
from torch import optim

model = ResNet18()

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)

## ResNet 18 Modificado

In [ ]:
from torch import nn
from torch import optim

model = ResNet18()

criterion = nn.BCELoss()

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)

## AdvancedCNN

In [ ]:
from torch import nn
from torch import optim

model = AdvancedCNN()

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)

## DenseNet_121CNN

In [ ]:
from torch import nn
from torch import optim

model = DenseNet_121CNN()

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)

## DenseNet_121 Modificado

In [ ]:
from torch import nn
from torch import optim

model = DenseNet_121CNN()

criterion = nn.ReLU()

optimizer = optim.Adam(model.parameters(), lr=0.001)

run(exp_config, train_dataloader, val_dataloader, test_dataloader, model, criterion, optimizer)